In [1]:
import time
from collections import deque
from itertools import chain
import numpy as np
import torch
import sys
import os
import copy
from scipy import stats
import torch.nn as nn
from src.utils import get_argparser
from src.encoders_ICA import NatureCNN

import pandas as pd
import datetime
from src.lstm_attn import subjLSTM
from src.All_Architecture import combinedModel

# import torchvision.models.resnet_conv1D as models
# from tensorboardX import SummaryWriter

from src.graph_the_works_fMRI import the_works_trainer
import matplotlib.pyplot as plt
import nibabel as nib
import h5py
import math
from copy import  copy
import matplotlib.colors as colors

import torch.nn.utils.rnn as tn

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

# try to get a  QUADRORTX6000; 1080 doesn't have enough memory. a6000 not compatible.

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.init()

In [4]:
#load my data
with np.load('/om2/scratch/Fri/jsmentch/nat_img/sourcedata/data/HBN/brain/clean/for_dice/movieTP.npz', 'rb') as file:
    sub_list=file['arr_0']
    ses_list=file['arr_1']
    dx_list=file['arr_2']
    mmp_data=file['arr_3']

In [5]:
#subject, parcels, timepoints
mmp_data.shape

(78, 360, 250)

In [6]:
dx_list.sum()
dx_list.shape

#37 with ASC; 41 NT; 78 total

(78,)

In [7]:
def find_indices_of_each_class(dx_list):
    HC_index = (dx_list == 0).nonzero()[0]
    SZ_index = (dx_list == 1).nonzero()[0]

    return HC_index, SZ_index

In [8]:
parser = get_argparser()
#args = parser.parse_args()

#args = parser.parse_args(args=['--req_1', '10', '--req_2', '10'])

args = parser.parse_args(args=['--path', '/om2/scratch/Fri/jsmentch/dice_scratch/wandb', 
                              '--oldpath', '/om2/scratch/Fri/jsmentch/dice_scratch/wandb', 
                              '--fig-path', '/om2/scratch/Fri/jsmentch/dice_scratch/wandb', 
                              '--p-path', '/om2/scratch/Fri/jsmentch/dice_scratch/wandb'])
                              #'--lstm_size', '360'])

In [9]:
start_time = time.time()

# ID = args.script_ID + 3
# ID = args.script_ID - 1 #? task array id...
JobID = args.job_ID #? used for naming files, it is set to 1


ID = 4 #? this seems to set which "gain" we use?
print('ID = ' + str(ID))
print('exp = ' + args.exp)
print('pretraining = ' + args.pre_training)
sID = str(ID)
currentDT = datetime.datetime.now()
d1 = currentDT.strftime("%Y-%m-%d%H:%M:%S")
d2 = '_' + str(JobID) + '_ startFold_' + str(args.starting_test_fold) + '_' + str(args.n_test_folds_to_run)

Name = args.exp + '_FBIRN_' + args.pre_training + 'DICE_Default'
dir = 'run-' + d1 + d2 + Name
dir = dir + '-' + str(ID)
wdb = 'wandb_new'

wpath = os.path.join(os.getcwd(), wdb)
path = os.path.join(wpath, dir)
args.path = path
os.mkdir(path)

wdb1 = 'wandb_new'
wpath1 = os.path.join(os.getcwd(), wdb1)


p = 'UF'
dir = 'run-2019-09-1223:36:31' + '-' + str(ID) + 'FPT_ICA_COBRE'
p_path = os.path.join(os.getcwd(), p)
p_path = os.path.join(p_path, dir) 
args.p_path = p_path
# os.mkdir(fig_path)
# hf = h5py.File('../FBIRN_AllData.h5', 'w')
tfilename = str(JobID) + 'outputFILENEWONE' + Name + str(ID)

output_path = os.path.join(os.getcwd(), 'Output')
output_path = os.path.join(output_path, tfilename)
# output_text_file = open(output_path, "w+")
# writer = SummaryWriter('exp-1')
ntrials = args.ntrials
# ngtrials = 10 #? seems unused?
# best_auc = 0. #? seems unused?
# best_gain = 0 #? seems unused?
current_gain=0
# train_sub_SZ = [15, 25, 50, 75, 142, 125] #142, 132 80 #? i think tr means train not TR ?
# train_sub_HC = [15, 25, 50, 75, 134, 125] #134, 124 74 #? so training on 142/134 .. ah

# With 16 per sub val, 10 WS working, MILC default
if args.exp == 'FPT':
    gain = [0.45, 0.05, 0.05, 0.15, 0.85]  # FPT
elif args.exp == 'UFPT':
    gain = [3, 3, 3, 3, 3, 3]  # UFPT
else:
    gain = [1, 1, 1, 1, 2.25, 1]  # NPT

ID = 4
exp = NPT
pretraining = None


In [10]:
#we will do a stratified kfold split

In [11]:
# sub_per_class_SZ = train_sub_SZ[ID]
# sub_per_class_HC = train_sub_HC[ID]
current_gain = gain[ID]
args.gain = current_gain
# sample_x = 100 #? unused ?
sample_y = 1 #? how many time points to use?
subjects = sub_list.shape[0] #78 #311

# tc = 160 #? unused ?

# samples_per_subject = int(tc / sample_y)
samples_per_subject = mmp_data.shape[2]
# samples_per_subject = int((tc - sample_y)+1)
# ntest_samples_perclass_SZ = 9
# ntest_samples_perclass_HC = 8
# if ID == 5:
#     nval_samples_perclass = 15
# else:
#     nval_samples_perclass_SZ = 9
#     nval_samples_perclass_HC = 8
# test_start_index = 0
# test_end_index = test_start_index + ntest_samples_perclass_SZ
window_shift = 1

if torch.cuda.is_available():
    cudaID = str(torch.cuda.current_device())
    print(torch.cuda.device_count())
    device = torch.device("cuda:0")
    device2 = torch.device("cuda:0")
    # device = torch.device("cuda:" + str(args.cuda_id))
else:
    device = torch.device("cpu")
    device2 = device
print('device = ', device)
print('device = ', device2)

# n_good_comp = 53
n_regions = 360

# with open('../DataandLabels/FBIRN_alldata_new_160.npz', 'rb') as file:
#     data = np.load(file)

data=mmp_data
# data[data != data] = 0 #??

for t in range(subjects):
    for r in range(n_regions):
        data[t, r, :] = stats.zscore(data[t, r, :])

data=np.nan_to_num(data) #get rid of NaNs because for some reason there are 250 nans?

1
device =  cuda:0
device =  cuda:0


In [12]:
# data = data + 2
data = torch.from_numpy(data).float()
finalData = np.zeros((subjects, samples_per_subject, n_regions, sample_y))
for i in range(subjects):#? I think it is just reshaping and not doing any shifting?
    for j in range(samples_per_subject):
        #if j != samples_per_subject-1:
        finalData[i, j, :, :] = data[i, :, (j * window_shift):(j * window_shift) + sample_y]
        #else:
            #finalData[i, j, :, :17] = data[i, :, (j * window_shift):]


start_path = '../Atlases2'
# count = 0; #? unused?

# test_indices_HC = [0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136]
# test_indices_SZ = [0, 9, 18, 27, 36, 45, 54, 63, 72, 81, 90, 99, 108, 117, 126, 135, 144, 153]


number_of_test_folds_to_run = args.n_test_folds_to_run
n_regions_output = n_regions
# tc_after_encoder = 155 #? unused?
HC_index, SZ_index = find_indices_of_each_class(dx_list)
print(HC_index.shape)
print(SZ_index.shape)


(41,)
(37,)


In [13]:
# return
# total_test_size = ntest_samples_perclass_HC + ntest_samples_perclass_SZ
results = torch.zeros(ntrials * number_of_test_folds_to_run, 10)
result_counter = 0

In [14]:
args

Namespace(pre_training='None', fMRI_twoD=False, deep=False, path='/net/vast-storage.ib.cluster/scratch/scratch/Fri/jsmentch/DICE/scripts/wandb_new/run-2023-01-0514:42:57_1_ startFold_0_1NPT_FBIRN_NoneDICE_Default-4', oldpath='/om2/scratch/Fri/jsmentch/dice_scratch/wandb', fig_path='/om2/scratch/Fri/jsmentch/dice_scratch/wandb', p_path='/net/vast-storage.ib.cluster/scratch/scratch/Fri/jsmentch/DICE/scripts/UF/run-2019-09-1223:36:31-4FPT_ICA_COBRE', exp='NPT', gain=2.25, temperature=0.25, script_ID=1, n_test_folds_to_run=1, starting_test_fold=0, teststart_ID=1, job_ID=1, ntrials=10, sample_number=0, env_name='MontezumaRevengeNoFrameskip-v4', num_frame_stack=1, no_downsample=True, pretraining_steps=100000, probe_steps=50000, num_processes=8, method='graph_the_works', linear=True, use_multiple_predictors=False, lr=0.0002, batch_size=5, epochs=300, cuda_id=1, seed=42, encoder_type='Nature', model_type='graph_the_works', feature_size=2, fully_connected=False, feature_size_pre_training=32, fM

In [15]:
#%debug
skf = StratifiedKFold(n_splits=5)
for test_ID, (trainval_indices, test_indices) in enumerate(skf.split(data, y=dx_list)):
    # print('train',trainval_indices)
    # print('test',test_indices)
    # test_ID = test_ID + args.starting_test_fold
#     if test_ID == 17:
#         ntest_samples_perclass_SZ = 7
#         ntest_samples_perclass_HC = 14#15#14

#         sub_per_class_SZ = 144#134
#         sub_per_class_HC = 128#118

    print('test Id =', test_ID)

    # test_start_index_SZ = test_indices_SZ[test_ID]
    # test_start_index_HC = test_indices_HC[test_ID]
    # test_end_index_SZ = test_start_index_SZ + ntest_samples_perclass_SZ
    # test_end_index_HC = test_start_index_HC + ntest_samples_perclass_HC
    # total_HC_index_tr_val = torch.cat([HC_index[:test_start_index_HC], HC_index[test_end_index_HC:]])
    # total_SZ_index_tr_val = torch.cat([SZ_index[:test_start_index_SZ], SZ_index[test_end_index_SZ:]])

    # HC_index_test = HC_index[test_start_index_HC:test_end_index_HC]
    # SZ_index_test = SZ_index[test_start_index_SZ:test_end_index_SZ]

    # total_HC_index_tr = total_HC_index_tr_val[:(total_HC_index_tr_val.shape[0] - nval_samples_perclass_HC)]
    # total_SZ_index_tr = total_SZ_index_tr_val[:(total_SZ_index_tr_val.shape[0] - nval_samples_perclass_SZ)]

    # HC_index_val = total_HC_index_tr_val[(total_HC_index_tr_val.shape[0] - nval_samples_perclass_HC):]
    # SZ_index_val = total_SZ_index_tr_val[(total_SZ_index_tr_val.shape[0] - nval_samples_perclass_SZ):]

    #auc_arr = torch.zeros(ngtrials, 1) #? seems unused?
    #avg_auc = 0. #? seems unused?
    
    sss = StratifiedShuffleSplit(n_splits=ntrials, test_size=0.2, random_state=0)
    for trial, (train_indices, val_indices) in enumerate(sss.split(X=data[trainval_indices], y=dx_list[trainval_indices] )):

        # print(val_indices)
        print ('trial = ', trial)

        g_trial=1
        output_text_file = open(output_path, "a+")
        output_text_file.write("Test fold number = %d Trial = %d\r\n" % (test_ID,trial))
        output_text_file.close()
        # Get subject_per_class number of random values
        # HC_random = torch.randperm(total_HC_index_tr.shape[0])
        # SZ_random = torch.randperm(total_SZ_index_tr.shape[0])
        # HC_random = HC_random[:sub_per_class_HC]
        # SZ_random = SZ_random[:sub_per_class_SZ]
        # HC_random = torch.randint(high=len(total_HC_index_tr), size=(sub_per_class,))
        # SZ_random = torch.randint(high=len(total_SZ_index_tr), size=(sub_per_class,))
        #

        # Choose the subject_per_class indices from HC_index_val and SZ_index_val using random numbers

        # HC_index_tr = total_HC_index_tr[HC_random]
        # SZ_index_tr = total_SZ_index_tr[SZ_random]

#         tr_index = torch.cat((HC_index_tr, SZ_index_tr))
#         val_index = torch.cat((HC_index_val, SZ_index_val))
#         test_index = torch.cat((HC_index_test, SZ_index_test))

#         tr_index = tr_index.view(tr_index.size(0))
#         val_index = val_index.view(val_index.size(0))
#         test_index = test_index.view(test_index.size(0))
        
        tr_index = torch.tensor(train_indices)
        val_index = torch.tensor(val_indices)
        test_index = torch.tensor(test_indices)

        
        tr_eps = torch.tensor(finalData[tr_index.long(), :, :, :],dtype=torch.float32)
        val_eps = torch.tensor(finalData[val_index.long(), :, :, :],dtype=torch.float32)
        test_eps = torch.tensor(finalData[test_index.long(), :, :, :],dtype=torch.float32)

        tr_labels = torch.tensor(dx_list[tr_index.long()])
        val_labels = torch.tensor(dx_list[val_index.long()])
        test_labels = torch.tensor(dx_list[test_index.long()])

        tr_labels = tr_labels.to(device)
        val_labels = val_labels.to(device)
        test_labels = test_labels.to(device)


        tr_eps = tr_eps.to(device)
        # val_eps = val_eps.to(device)
        test_eps = test_eps.to(device)

        print('tr_eps',tr_eps.shape,tr_eps.dtype)
        print('val_eps',val_eps.shape,val_eps.dtype)
        print('test_eps',test_eps.shape,test_eps.dtype)

        print('tr_labels',tr_labels.shape,tr_labels.dtype)
        print('val_labels',val_labels.shape,val_labels.dtype)
        print('test_labels',test_labels.shape,test_labels.dtype)
        
        
        
        
        

        
        
        

        observation_shape = finalData.shape
        L=""
        lmax=""
        number_of_graph_channels = 1
        if args.model_type == "graph_the_works":
            print('obs shape',observation_shape[3])
            encoder = NatureCNN(observation_shape[3], args)

            encoder.to(device)
            lstm_model = subjLSTM(device, sample_y, args.lstm_size, num_layers=args.lstm_layers,
                                  freeze_embeddings=True, gain=current_gain,bidrection=True)
            dir = ""
            if args.pre_training == "DECENNT":
                dir = 'Pre_Trained/DECENNT/UsingHCP500TP/model.pt'
                args.oldpath = wpath1 + '/Pre_Trained/DECENNT/UsingHCP500TP'





        complete_model = combinedModel(encoder,lstm_model, samples_per_subject, gain=current_gain, PT=args.pre_training, exp=args.exp, device_one=device, oldpath=args.oldpath,n_regions=n_regions,device_two=device2,device_zero=device2,device_extra=device2 )
        complete_model.to(device)
        config = {}
        config.update(vars(args))
        # print("trainershape", os.path.join(wandb.run.dir, config['env_name'] + '.pt'))
        config['obs_space'] = observation_shape  # weird hack
        if args.method == "graph_the_works":
            trainer = the_works_trainer(complete_model, config, device=device2, device_encoder=device,
                                        tr_labels=tr_labels,
                                  val_labels=val_labels, test_labels=test_labels, trial=str(trial),
                                        crossv=str(test_ID),gtrial=str(g_trial))

        else:
            assert False, "method {} has no trainer".format(args.method)
        results[result_counter][0], results[result_counter][1], results[result_counter][2], \
        results[result_counter][3],results[result_counter][4],\
        results[result_counter][5], _ = trainer.train(tr_eps, val_eps, test_eps)

        
        #ipdb>  print(batch_sizes.dtype)
        #torch.int64
        #ipdb>  print(input.dtype)
        #torch.float64
        result_counter = result_counter + 1
        tresult_csv = os.path.join(args.path, 'test_results' + sID + '.csv')
        np.savetxt(tresult_csv, results.numpy(), delimiter=",")

np_results = results.numpy()
auc = np_results[:,1]
acc = np_results[:, 0]
print('mean test auc = ', np.mean(acc[:]))
print('mean test acc = ', np.mean(auc[:]))

tresult_csv = os.path.join(args.path, 'test_results' + sID + '.csv')
np.savetxt(tresult_csv, np_results, delimiter=",")

elapsed = time.time() - start_time
print('total time = ', elapsed);


# ipdb>  input.shape
# torch.Size([5, 129600]) ### 129600 is 360^2
# ipdb>  weight.t().shape
# torch.Size([10000, 64])

test Id = 0
trial =  0
tr_eps torch.Size([49, 250, 360, 1]) torch.float32
val_eps torch.Size([13, 250, 360, 1]) torch.float32
test_eps torch.Size([16, 250, 360, 1]) torch.float32
tr_labels torch.Size([49]) torch.int64
val_labels torch.Size([13]) torch.int64
test_labels torch.Size([16]) torch.int64
obs shape 1
lstm init weight
lr =  0.0002
2.25
initNPT
e 0
tr_eps torch.Size([49, 250, 360, 1])
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 1, 0, 1, 0], device='cuda:0')
mode train
self.device cuda:0
epoch 0
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 1, 1, 0, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 0
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 1, 0, 0, 0], device='cuda:0')
mode train
self.device cuda:0
e

/om2/user/jsmentch/anaconda/envs/dice/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 1, 0, 0, 0], device='cuda:0')
mode train
self.device cuda:0
epoch 19
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 1, 1, 0, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 19
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 1, 1, 0, 0], device='cuda:0')
mode train
self.device cuda:0
epoch 19
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 1, 1, 0, 0], device='cuda:0')
mode train
self.device cuda:0
epoch 19
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 1, 1, 0, 1], device='cuda:0')
mode train
sel

/om2/user/jsmentch/anaconda/envs/dice/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 0, 1, 1, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 20
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 0, 1, 0, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 20
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 1, 0, 0, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 20
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 0, 1, 0, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 20
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 1, 1, 0, 1], device='cuda:0')
mode train
sel

...

/om2/user/jsmentch/anaconda/envs/dice/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 0, 1, 1, 0], device='cuda:0')
mode train
self.device cuda:0
epoch 28
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 1, 0, 1, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 28
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 1, 0, 1, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 28
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 0, 0, 1, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 28
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 1, 1, 1, 0], device='cuda:0')
mode train
sel

/om2/user/jsmentch/anaconda/envs/dice/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 0, 0, 1, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 31
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 0, 1, 0, 0], device='cuda:0')
mode train
self.device cuda:0
epoch 31
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([1, 0, 0, 1, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 31
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 1, 1, 1, 1], device='cuda:0')
mode train
self.device cuda:0
epoch 31
logits, kl_loss, FC, FC_temporal = self.model(sx, targets, mode, self.device, epoch)
sx torch.Size([5, 250, 360, 1])
targets tensor([0, 0, 1, 1, 1], device='cuda:0')
mode train
sel

KeyboardInterrupt: 

In [ ]:
complete_model

In [ ]:
print(tr_eps.dtype)
print(val_eps.dtype)
print(test_eps.dtype)

print(tr_labels.dtype)
print(val_labels.dtype)
print(test_labels.dtype)